# Day 4: Passport Processing
https://adventofcode.com/2020/day/4

In [1]:
inputLines = lines <$> readFile "input/day04.txt"

In [2]:
testInput = [ "ecl:gry pid:860033327 eyr:2020 hcl:#fffffd"
            , "byr:1937 iyr:2017 cid:147 hgt:183cm"
            , ""
            , "iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884"
            , "hcl:#cfa07d byr:1929"
            , ""
            , "hcl:#ae17e1 iyr:2013"
            , "eyr:2024"
            , "ecl:brn pid:760753108 byr:1931"
            , "hgt:179cm"
            , ""
            , "hcl:#cfa07d eyr:2025 pid:166559648"
            , "iyr:2011 ecl:brn hgt:59in"]

In [3]:
import Data.List.Split (splitOn)  -- install with 'stack install split'
import Data.List (intercalate)

Split input on blank lines to get the individual passports:

In [4]:
passports = splitOn "\n\n" . intercalate "\n"

In [5]:
passports testInput

["ecl:gry pid:860033327 eyr:2020 hcl:#fffffd\nbyr:1937 iyr:2017 cid:147 hgt:183cm","iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884\nhcl:#cfa07d byr:1929","hcl:#ae17e1 iyr:2013\neyr:2024\necl:brn pid:760753108 byr:1931\nhgt:179cm","hcl:#cfa07d eyr:2025 pid:166559648\niyr:2011 ecl:brn hgt:59in"]

Represent a passport as a map:

In [6]:
import qualified Data.Map as Map

In [7]:
parsePassport :: String -> Map.Map String String
parsePassport = Map.fromList
              . map ((\ [k, v] -> (k, v)) . splitOn ":")
              . words

# Part 1
All fields except `cid` are required:

In [8]:
required = ["byr", "iyr", "eyr", "hgt", "hcl", "ecl", "pid"]

In [9]:
checkPassport passport = all ((/= Nothing). (`Map.lookup` passport)) required

In [10]:
countValidPassports1 = length . filter id . map (checkPassport . parsePassport)

Verify given result for test input:

In [11]:
countValidPassports1 . passports $ testInput

2

## Solution, part 1

In [12]:
countValidPassports1 . passports <$> inputLines

228

# Part 2
For all fields that contain years, we have to compare the value with the minimal and maximal accepted year.

In [13]:
intValidator :: Int -> Int -> String -> Bool
intValidator minValue maxValue value = minValue <= number && number <= maxValue
    where
        number = read value

Hair color, eye color, and passport ID are best validated with regular expressions.

In [14]:
import Text.Regex.PCRE  -- install with 'stack install regex-pcre'

In [15]:
regexValidator :: String -> String -> Bool
regexValidator pattern value = (value =~ pattern)

To validate the height, we parse the value and the unit wit a regular expression. If this is successful, we compare the height with the min and max value for the respective unit.

In [16]:
heightValidator :: String -> Bool
heightValidator value = 
    case heightAndUnit of 
        Nothing             -> False
        Just (height, unit) -> minHeight unit <= height && height <= maxHeight unit
    where
        heightAndUnit = do
            [[_, heightValue, unit]] <- value =~~ "^([0-9]+)(cm|in)$"
            return (read heightValue, unit) 

        minHeight "cm" = 150
        minHeight "in" = 59

        maxHeight "cm" = 193
        maxHeight "in" = 76

Assign the correct validator to each passport field.

In [17]:
validators :: [(String, String -> Bool)]
validators = [
    ("byr", intValidator 1920 2002),
    ("iyr", intValidator 2010 2020),
    ("eyr", intValidator 2020 2030),
    ("hgt", heightValidator),
    ("hcl", regexValidator "^#[0-9a-f]{6}$"),
    ("ecl", regexValidator "^(amb|blu|brn|gry|grn|hzl|oth)$"),
    ("pid", regexValidator "^[0-9]{9}$")]

A passport is valid if all fields are valid.

In [18]:
validatePassport passport = all 
    ((== Just True) . 
     (\ (field, validator) -> validator <$> field `Map.lookup` passport)) validators

In [19]:
countValidPassports2 = length . filter id . map (validatePassport . parsePassport)

## Solution, part 2

In [20]:
countValidPassports2 . passports <$> inputLines

175

# Appendix

`validatePassport` can be written without a lambda.

In [21]:
validatePassport' passport = all 
    ((== Just True) . 
     uncurry (flip (<$>) . (`Map.lookup` passport))) validators

In [22]:
countValidPassports2' = length . filter id . map (validatePassport . parsePassport)

In [23]:
countValidPassports2' . passports <$> inputLines

175